In [8]:
from datasets import load_dataset

datasets_path = "./datasets/"

datasets = load_dataset(path=datasets_path) # 会自动切分 train 和 test，这里提前选了 train 后面就不用选了

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 79692 examples [00:03, 22159.94 examples/s]


In [12]:
print(type(datasets['train']))
print(datasets['train'][0])

<class 'datasets.arrow_dataset.Dataset'>
{'instruction': '你是一个精通法律的专家，请根据用户的问题给出专业的回答。', 'input': '违章停车与违法停车是否有区别？', 'output': '对违反道路交通安全法律、法规关于机动车停放、临时停车规定的，可以指出违法行为，并予以口头警告，令其立即驶离。机动车驾驶人不在现场或者虽在现场但拒绝立即驶离，妨碍其他车辆、行人通行的处二十元以上二百元以下罚款。现在人们大多是称作违法停车，因此在法律责任上也会更多一些，不要以为违反交通规章制度问题不大，不要认为违法停车是罚款而已。'}


In [17]:
# sft
import torch
import argparse
# import swanlab
import pandas as pd
from peft import LoraConfig, prepare_model_for_kbit_training
# from swanlab.integration.transformers import SwanLabCallback
# import bitsandbytes as bnb # 需要在 GPU 环境下才能正确导入

from transformers import TrainingArguments
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig, 
    DataCollatorForSeq2Seq
    )

from datasets import load_dataset, Dataset
# from trl import SFTTrainer

def process_func(example):
    TEMPLATE = '<|im_start|>system\n{}<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n<|im_start|>assistant\n'
    MAX_LENGTH = 1024

    input_ids, attention_mask, labels = [], [], []

    instruction = tokenizer(
        TEMPLATE.format(example['instruction'], example['input']),
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['output']}", add_special_tokens=False)

    input_ids = (
        instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    )
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = (
        [-100] * len(instruction["input_ids"])
        + response["input_ids"]
        + [tokenizer.pad_token_id]
    )

    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
        
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

model_path = "/data/qwen/Qwen2___5-7B-Instruct/"
tokenizer = AutoTokenizer.from_pretrained(
    model_path, 
    use_fast=False, 
    trust_remote_code=True,         # If the model is defined by a remote code, trust it
) 



In [18]:
datasets_path = "./datasets/DISC-Law-SFT-Pair-QA-released.jsonl"
train_dataset = pd.read_json(datasets_path, lines=True)[10:10000]
test_dataset = pd.read_json(datasets_path, lines=True)[:10]

train_dataset = Dataset.from_pandas(train_dataset)


In [20]:
model_path = "/data/qwen/Qwen2___5-7B-Instruct/"
tokenizer = AutoTokenizer.from_pretrained(
    model_path, 
    use_fast=False, 
    trust_remote_code=True,         # If the model is defined by a remote code, trust it
) 

train_dataset = train_dataset.map(
    process_func, 
    remove_columns=train_dataset.column_names
    )


Map: 100%|██████████| 9990/9990 [02:18<00:00, 71.97 examples/s] 


In [22]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

lora_path = "/data/workbench/checkpoints/"
base_path = "/data/qwen/Qwen2___5-7B-Instruct"

base_model = AutoModelForCausalLM.from_pretrained(
    base_path, 
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    )

model = PeftModel.from_pretrained(base_model, lora_path)
model = model.merge_and_unload()

# model = AutoModelForCausalLM.from_pretrained("/data/qwen/Qwen2___5-7B-Instruct/")
tokenizer = AutoTokenizer.from_pretrained(lora_path)

INSTRUCTION = "你是一个精通法律的专家，请根据用户的问题给出专业的回答。"
while True:
    prompt = []
    prompt.append({"role": "system", "content": INSTRUCTION})

    print("--------------------")
    question = input('User: ' + '\n')

    prompt.append({"role": "user", "content": question})

    input_text = tokenizer.apply_chat_template(
            prompt,
            tokenize=False,
            add_generation_prompt=True
        )
    model_inputs = tokenizer([input_text], return_tensors="pt").to(model.device)

    if model_inputs.input_ids.size()[1]>32000:
        break

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        pad_token_id=tokenizer.eos_token_id
    )

    if len(generated_ids)>114514:
        break

    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print('Assistant:\n')
    print(response)

    print("--------------------")
    print('\n')




In [3]:
import pandas as pd
df = pd.read_parquet('/data/workbench/datasets/train-00000-of-00001.parquet',  engine='pyarrow')  # 本地文件
type(df)

pandas.core.frame.DataFrame

In [4]:
new_columns = ["prompt", "chosen", "rejected"]
df.columns = new_columns
df['chosen'][0]

'（参考诉讼与非诉讼程序法-仲裁法2017-09-01：第十七条（一））\n\n根据《诉讼与非诉讼程序法-仲裁法2017-09-01》第十七条（一）规定，如果约定的仲裁事项超出了法律规定的仲裁范围，仲裁协议将会无效。因此，乙公司不能简单认为仲裁协议无效，而需要解释和证明合同纠纷是否超出了法律规定的仲裁范围。\n诉讼与非诉讼程序法-仲裁法2017-09-01:    "中国仲裁协会依照本法和民事诉讼法的有关规定制定仲裁规则。",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "第十六条 仲裁协议包括合同中订立的仲裁条款和以其他书面方式在纠纷发生前或者纠纷发生后达成的请求仲裁的协议。",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "仲裁协议应当具有下列内容：",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "（一）请求仲裁的意思表示；",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "（二）仲裁事项；",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "（三）选定的仲裁委员会。",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "第十七条 有下列情形之一的，仲裁协议无效：",\n 诉讼与非诉讼程序法-仲裁法2017-09-01:    "（一）约定的仲裁事项超出法律规定的仲裁范围的；",\n'

In [5]:
import pandas as pd

def is_valid_paragraph(para: str) -> bool:
    """验证单个自然段是否满足条件"""
    stripped = para.strip()
    # 必须同时满足：非空字符串 + 以句号结尾
    return bool(stripped) and stripped.endswith("。")

def process_text(text: str) -> str:
    """
    处理文本的核心逻辑
    参数：
        text : 原始文本（用换行符分隔的自然段）
    返回：
        处理后的文本，保留原段落结构
    """
    # 分割自然段并保留原始换行结构
    original_paragraphs = text.split('\n')
    
    # 过滤有效段落
    valid_paragraphs = []
    for para in original_paragraphs:
        # 保留原始段落格式，仅过滤不符合条件的
        if is_valid_paragraph(para):
            valid_paragraphs.append(para)
    
    # 重新组合有效段落
    return '\n'.join(valid_paragraphs)

def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """清洗DataFrame中的指定列"""
    for col in ['chosen', 'rejected']:
        df[col] = df[col].astype(str).apply(process_text)
    return df

# ----------------------------

In [6]:
df = clean_dataframe(df)

In [12]:
df = pd.read_csv("/data/workbench/datasets/law-gpt.csv")

In [14]:
len(df)

500

In [13]:
import pandas as pd
from datasets import load_dataset, Dataset

def process_func(examples):
    prompt, chosen, rejected = [], [], []

    text = f"<|im_start|>user\n{examples['prompt']}<|im_end|>\n<|im_start|>assistant\n"
    prompt.append(text)

    # assert examples["chosen"][i] == "assistant"
    text = f"{examples['chosen']}<|im_end|>"
    chosen.append(text)

    # assert examples["rejected"][i] == "assistant"
    text = f"{examples['rejected']}<|im_end|>"
    rejected.append(text)

    result = {"prompt": prompt, "chosen": chosen, "rejected": rejected}
    return result

datasets_path = "./datasets/law-gpt.csv"
df = pd.read_csv(datasets_path)
bdf = pd.read_csv(datasets_path)
tdf = Dataset.from_pandas(df)
ttdf = tdf.map(
    process_func,
    remove_columns=train_dataset.column_names,
)


Map: 100%|██████████| 500/500 [00:00<00:00, 11289.70 examples/s]


In [12]:
bdf['prompt'][0]


'甲公司与乙公司签订了合同，其中包含仲裁条款，并选定了中国仲裁协会作为仲裁机构。当纠纷发生后，甲公司请求仲裁解决，但乙公司却表示仲裁协议无效，认为纠纷超出了法律规定的仲裁范围。这种情况下，仲裁协议是否有效？'

In [14]:
tdf[0]

{'Unnamed: 0': 0,
 'prompt': '甲公司与乙公司签订了合同，其中包含仲裁条款，并选定了中国仲裁协会作为仲裁机构。当纠纷发生后，甲公司请求仲裁解决，但乙公司却表示仲裁协议无效，认为纠纷超出了法律规定的仲裁范围。这种情况下，仲裁协议是否有效？',
 'chosen': '根据《诉讼与非诉讼程序法-仲裁法2017-09-01》第十七条（一）规定，如果约定的仲裁事项超出了法律规定的仲裁范围，仲裁协议将会无效。因此，乙公司不能简单认为仲裁协议无效，而需要解释和证明合同纠纷是否超出了法律规定的仲裁范围。',
 'rejected': '根据《民事诉讼法》相关规定，对依法设立的仲裁机构的裁决，一方当事人不履行的，对方当事人可以向有管辖权的人民法院申请执行。受申请的人民法院应当执行。被申请人提出证据证明仲裁裁决不应该执行的，经人民法院组成合议庭审查核实，可以裁定不予执行。但是，如果当事人在合同中没有订有仲裁条款或者事后没有达成书面仲裁协议，或者裁决的事项不属于仲裁协议的范围或者仲裁机构无权仲裁的，裁定依然应当执行。因此，在本案中，甲方可以向有管辖权的人民法院申请执行仲裁机构的裁决，乙方应当履行该裁决书。'}

In [16]:
ttdf[0]

{'prompt': ['<|im_start|>user\n甲公司与乙公司签订了合同，其中包含仲裁条款，并选定了中国仲裁协会作为仲裁机构。当纠纷发生后，甲公司请求仲裁解决，但乙公司却表示仲裁协议无效，认为纠纷超出了法律规定的仲裁范围。这种情况下，仲裁协议是否有效？<|im_end|>\n<|im_start|>assistant\n'],
 'chosen': ['根据《诉讼与非诉讼程序法-仲裁法2017-09-01》第十七条（一）规定，如果约定的仲裁事项超出了法律规定的仲裁范围，仲裁协议将会无效。因此，乙公司不能简单认为仲裁协议无效，而需要解释和证明合同纠纷是否超出了法律规定的仲裁范围。<|im_end|>'],
 'rejected': ['根据《民事诉讼法》相关规定，对依法设立的仲裁机构的裁决，一方当事人不履行的，对方当事人可以向有管辖权的人民法院申请执行。受申请的人民法院应当执行。被申请人提出证据证明仲裁裁决不应该执行的，经人民法院组成合议庭审查核实，可以裁定不予执行。但是，如果当事人在合同中没有订有仲裁条款或者事后没有达成书面仲裁协议，或者裁决的事项不属于仲裁协议的范围或者仲裁机构无权仲裁的，裁定依然应当执行。因此，在本案中，甲方可以向有管辖权的人民法院申请执行仲裁机构的裁决，乙方应当履行该裁决书。<|im_end|>']}

In [17]:
from datasets import Dataset

# 检查数据集结构
print("数据集列结构:", ttdf.column_names)
print("样本示例:", ttdf[0])

# 检查数据类型一致性
for col in ["prompt", "chosen", "rejected"]:
    print(f"\n检查列 '{col}':")
    sample_types = [type(x) for x in ttdf[col]]
    unique_types = set(sample_types)
    print(f"存在 {len(unique_types)} 种类型: {unique_types}")
    if list in unique_types:
        print("警告：检测到列表类型！DPOTrainer需要字符串类型字段")

数据集列结构: ['prompt', 'chosen', 'rejected']
样本示例: {'prompt': ['<|im_start|>user\n甲公司与乙公司签订了合同，其中包含仲裁条款，并选定了中国仲裁协会作为仲裁机构。当纠纷发生后，甲公司请求仲裁解决，但乙公司却表示仲裁协议无效，认为纠纷超出了法律规定的仲裁范围。这种情况下，仲裁协议是否有效？<|im_end|>\n<|im_start|>assistant\n'], 'chosen': ['根据《诉讼与非诉讼程序法-仲裁法2017-09-01》第十七条（一）规定，如果约定的仲裁事项超出了法律规定的仲裁范围，仲裁协议将会无效。因此，乙公司不能简单认为仲裁协议无效，而需要解释和证明合同纠纷是否超出了法律规定的仲裁范围。<|im_end|>'], 'rejected': ['根据《民事诉讼法》相关规定，对依法设立的仲裁机构的裁决，一方当事人不履行的，对方当事人可以向有管辖权的人民法院申请执行。受申请的人民法院应当执行。被申请人提出证据证明仲裁裁决不应该执行的，经人民法院组成合议庭审查核实，可以裁定不予执行。但是，如果当事人在合同中没有订有仲裁条款或者事后没有达成书面仲裁协议，或者裁决的事项不属于仲裁协议的范围或者仲裁机构无权仲裁的，裁定依然应当执行。因此，在本案中，甲方可以向有管辖权的人民法院申请执行仲裁机构的裁决，乙方应当履行该裁决书。<|im_end|>']}

检查列 'prompt':
存在 1 种类型: {<class 'list'>}
警告：检测到列表类型！DPOTrainer需要字符串类型字段

检查列 'chosen':
存在 1 种类型: {<class 'list'>}
警告：检测到列表类型！DPOTrainer需要字符串类型字段

检查列 'rejected':
存在 1 种类型: {<class 'list'>}
警告：检测到列表类型！DPOTrainer需要字符串类型字段
